# La reconnaissance d'objets par Yolo
Exemple pris d'une photo de voie rapide avec reconnaissance de bus et voitures

Le modèle utilisé est le yolov3
Dans ce modèle les fichiers ci-dessous sont utilisés:
- yolo/yolov3.weights
- yolo/yolov3.cfg
- yolo/coco.names

Le 1er comporte l'information des Wi du réseau de neurone préentraîné avec 80 classes d'objets dont les bus, voitures. 
Le 2e représente le schéma du réseau de neurones et notamment les filtres de convolution construits.
Le 3e labellise et nomme ces 80 classes préentraînées et reconnues.


In [1]:
import cv2
import numpy as np
import torch

# Charger les fichiers de configuration et de poids du modèle YOLO
net = cv2.dnn.readNet("yolo/yolov3.weights", "yolo/yolov3.cfg")
layer_names = net.getLayerNames()

print("--- len(layer_names) ---")
print(len(layer_names))

print("--- type(layer_names) ---")
print(type(layer_names))

print("--- layer_names ---")
print(layer_names)

print("--- nb couches de sortie du modèle YOLO ---")
print(len(net.getUnconnectedOutLayers()))

print("--- type des couches de sortie du modèle YOLO ---")
print(type(net.getUnconnectedOutLayers()))

output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

# Charger les noms des classes (objets que YOLO peut détecter)
with open("yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Charger l'image à traiter
image = cv2.imread('bus-voitures.jpg')  # Remplacez 'image.jpg' par le chemin de votre image
height, width, channels = image.shape

print("--- conversion de l'image ---")

# Convertir l'image en un format adapté pour YOLO (un blob)
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

print("--- détection d'objets ---")

outs = net.forward(output_layers)

# Liste pour stocker les sous-images avec leur étiquette et confiance
detected_images = []

print("--- Analyse des résultats de la détection ---")

# Analyser les résultats de la détection
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Seuil de confiance
            # Extraire les coordonnées des boîtes englobantes
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            # Calculer les coordonnées du coin supérieur gauche de la boîte
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

print("--- suppression non maximale pour éliminer les doublons ---")

# Appliquer la suppression non maximale pour éliminer les doublons (overlapping boxes)
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Afficher les résultats sur l'image
for i in range(len(boxes)):
    if i in indices:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i], 2))
        color = (0, 255, 0)  # Vert
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label + " " + confidence, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

print("--- Afficher l'image avec les objets détectés et leur étiquette ---")

# Afficher l'image avec les objets détectés
cv2.imshow("bus-voitures", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

--- len(layer_names) ---
254
--- type(layer_names) ---
<class 'tuple'>
--- layer_names ---
('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_

# Cette cellule est un exemple de conversion de coordonnées spatiales en des données normalisées au format yolo.

Les coordonnées en entrée sont celles d'une boîte englobante (x_min, y_min, x_max, y_max) typique yolo
Il est nécessaire également de connaître la taille de l'image (width_image, height_image)

La fonction coord_norm retourne les coordonnées normalisées de YOLO (center_x, center_y, width, height).

En entrée le fichier ayant plusieurs coordonnées de rectangles dans l'image, chacu définissant un objet labellisé,
En 1ère ligne la taille de l'image
Et les lignes suivantes et jusqu'à la fin les coordonnées (x_min, y_min, x_max, y_max) des rectangles (appelés boîtes englobantes dans la sémantique Yolo).

A noter que dans une version plus récente de Yolo (après v3), le modèle gère les coordonnées devenant des segments; le rectangle devient un polygone à plusieurs sommets.

In [3]:
import csv

def coord_norm(width_image, height_image, x_min, y_min, x_max, y_max):
    # Calcul des coordonnées normalisées
    center_x = (x_min + x_max) / 2 / width_image
    center_y = (y_min + y_max) / 2 / height_image
    width = (x_max - x_min) / width_image
    height = (y_max - y_min) / height_image
    
    return center_x, center_y, width, height

classe_livre = '0'

# Lire le fichier classe csv et écriture dans le fichier classe txt

with open("images/train/livres.csv", mode='r', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    
    # Lire les en-têtes (première ligne)
    headers = next(csvreader)
    headers=[int(x) for x in headers]  # string transformés en int
    
    print(f"headers: {headers}")
    
    # Ouvrir un fichier texte pour l'écriture
    with open("images/train/livres.txt", mode='w', encoding='utf-8') as txtfile:
        
        for row in csvreader:
            print(f"row: {row}")
            row=[int(x) for x in row]  # string transformés en int
            coord = coord_norm(*headers, *row)
            print(f"coord:{coord}")
            coord=[str(x) for x in coord]  # string transformés en int
            txtfile.write(classe_livre + ',' + coord[0] + ',' + coord[1]+ ',' + coord[2]+ ',' + coord[3] + '\n')


headers: [4080, 3072]
row: ['572', '148', '980', '2434']
coord:(0.19019607843137254, 0.4202473958333333, 0.1, 0.744140625)
row: ['837', '342', '1343', '2310']
coord:(0.26715686274509803, 0.431640625, 0.12401960784313726, 0.640625)
row: ['1214', '328', '1697', '2434']
coord:(0.3567401960784314, 0.4495442708333333, 0.11838235294117647, 0.685546875)
row: ['1669', '352', '1948', '2350']
coord:(0.4432598039215686, 0.4397786458333333, 0.06838235294117648, 0.650390625)
row: ['1945', '345', '2148', '2350']
coord:(0.501593137254902, 0.4386393229166667, 0.04975490196078431, 0.6526692708333334)
row: ['2091', '430', '2366', '2333']
coord:(0.5462009803921568, 0.44970703125, 0.06740196078431372, 0.6194661458333334)
row: ['2295', '427', '2675', '2333']
coord:(0.6090686274509803, 0.44921875, 0.09313725490196079, 0.6204427083333334)
row: ['2559', '593', '2879', '2309']
coord:(0.6664215686274509, 0.4723307291666667, 0.0784313725490196, 0.55859375)
row: ['2770', '484', '3147', '2326']
coord:(0.7251225490

# Entraînement avec une photo

## Dans l'inventaire se trouvent divers objets non préentraînés par yolo.

C'est ce que nous allons faire ici par une photo d'une étagère avec des livres.
Nous prenons le modèle préentraîné de la version 8 de Yolo.

In [5]:
from ultralytics import YOLO
import cv2
import torch

# Charger l'image à prédire
image = cv2.imread("images/train/livres.jpg")

# Charger le modèle préexistant (ou choisir un modèle de base)
model = YOLO('yolov8s.yaml')  # ou 'yolov8s.yaml' pour YOLOv8

# Entraîner le modèle sur vos données
model.train(data='data.yaml', epochs=50, batch=16, imgsz=640)

# Sauvegarder le modèle après l'entraînement
model.save('my_yolov8_model.pt')


New https://pypi.org/project/ultralytics/8.3.49 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.3 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

train: Scanning C:\Users\Admin.local\Documents\7-Méthodologie MS\Projet Machine Learning\labels\train.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]
val: Scanning C:\Users\Admin.local\Documents\7-Méthodologie MS\Projet Machine Learning\labels\val.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]


Plotting labels to runs\detect\train25\labels.jpg... 


: 

La version des différentes librairies Yolo, Torch, Pytorch, Cuda fait que le programme tombe régulièrement en erreur à différents stades du projet. 

L'erreur rencontrée lors de la sauvegarde du modèle mentionne :
torch.save({**self.ckpt, **updates}, filename)
TypeError: 'NoneType' object is not a mapping

Un contournement possible est de faire appel à 
torch.save(model.state_dict(), 'my_yolov8s_model.pt')
au lieu de model.save('my_yolov8_model.pt')

In [1]:
# # Charger l'image à prédire
# image = cv2.imread("images/train/livres.jpg")


# Charger le modèle préexistant (ou choisir un modèle de base)
model = YOLO('yolov8s.yaml')  # ou 'yolov8s.yaml' pour YOLOv8

# Entraîner le modèle sur vos données
model.train(data='data.yaml', epochs=50, batch=16, imgsz=640)

# Sauvegarde inhabituelle
# remplacé par torch.save car le modèle n'ait pas encore généré avec un checkpoint valide au moment de la sauvegarde. 
torch.save(model.state_dict(), 'my_yolov8s_model.pt')



NameError: name 'YOLO' is not defined

Pour contourner le problème, la sauvegarde contient uniquement l'état du modèle (sans manipuler self.ckpt)

On lance maintenant une prédiction d'objet.
Le cas simple pris est celui de l'examen de la même photo.

Si l'on a fait une sauvegarde réussie:

In [ ]:

# Charger le modèle entraîné
model = YOLO('my_yolov8_model.pt')

# Charger une image pour la prédiction
image = cv2.imread("images/train/livres.jpg")

# Effectuer la prédiction
results = model(image)

# Afficher les résultats
results.show()  # Affiche l'image avec les annotations de détection
results.save()  # Sauvegarde l'image avec les annotations

Si la sauvegarde précédente n'a pas réussi, 
(rappel : car non génération d'un checkpoint valide au moment de la sauvegarde)
il faut charger également le schéma du modèle:

In [2]:
# Charger le modèle entraîné
model = YOLO('my_yolov8_model.pt')

# Recréer le modèle en utilisant la même configuration
model = YOLO('yolov8s.yaml')  # avec le fichier de configuration pour Yolov8
model.load_state_dict(torch.load('my_yolov8s_model.pt'))  # Charger les poids sauvegardés par torch.save

# Charger une image pour la prédiction
image = cv2.imread("images/train/livres.jpg")

# Effectuer la prédiction
results = model(image)

# Afficher les résultats
results.show()  # Affiche l'image avec les annotations de détection
results.save()  # Sauvegarde l'image avec les annotations

NameError: name 'YOLO' is not defined

Ne pas tenir compte du dernier message d'erreur (NameError: name 'YOLO' is not defined), celui-ci n'est qu'une exécution partielle de la dernière cellule. 


## Le code devrait s'exécuter.
Une mauvaise interaction entre les bibliothèques Yolo, Python (3.13 puis 3.12.7), Pytorch fait lever une exception à l'exécution de la cellule (et des suivantes) sous Jupyter.
Par ailleurs le code des dernères cellules marche partiellement (toujours pour la même raison).
Le calcul passe ou non selon le paramétrage de Cuda; ici sur W11 je ne réussis pas à lancer le traitement sur la GPU 

Une amélioration est prévue avec plusieurs axes à suivre:
* Augmentation du nombre d'images (train, val)
* Modification du paramétrage data.yaml (modèle yolov8)
